In [3]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Ignore Harmless Warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Data Augmentation for training dataset

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(), # Default probability of 0.5
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # Mean and Standard Deviation for each colour channels. Helps in faster convergence
                        [0.229, 0.224, 0.225])
])

# Data Augmentation for the testing dataset
# We dont need to perform random rotations or flips in the test dataset

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

root = 'C:\\Users\\shant\\CNN_Architectures_ImageNet_Pytorch\\Dataset\\CATS_DOGS\\'

# Location of the training data
train_data = datasets.ImageFolder(os.path.join(root,'train'),transform = train_transform)

# Location of the testing data
test_data = datasets.ImageFolder(os.path.join(root,'test'),transform = test_transform)

# Set a seed
torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10)

In [5]:
class GoogleNet(nn.Module):
    
    def __init__():
        
        super().__init__()
    
    def forward(self):
        pass

In [8]:
torch.cuda.memory_allocated()

0